In [ ]:
import os
import time
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI

# Retrieve API keys from local environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

# Initialize OpenAI and Pinecone clients
client = OpenAI(api_key=openai_api_key)
pc = Pinecone(api_key=pinecone_api_key)

# Define index and model
index_name = "events-index"
model = "text-embedding-3-small"
